In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import os
import random
from storysim import StorySimulator
import pandas as pd

/homes/ngetach/miniconda/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')


In [4]:
# This example is the new way to use the OpenAI lib for python
client = OpenAI()

def prompt_gpt(prompt, model):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    
    return response.choices[0].message.content

In [5]:
graph = {
        "hole_1": ["hole_2", "field"],
        "hole_2": ["hole_1", "hole_3"],
        "hole_3": ["hole_2", "hole_4"],
        "hole_4": ["hole_3", "field"],
        "field": ["hole_1", "hole_4"]
    }

events = {
        
        7:{"name": "cross_paths","actors": ["Alice", "Bob"], "location": "hole_2"}, 
        8: {"name": "exclusive_random", "actors":["Alice", "Bob"], "stop": 10 },
        13: {"name": "mislead", "actors":["Alice", "Bob"]}
    }


sim = StorySimulator(
        people=["Alice", "Bob", "Charlie", "Danny"],
        locations=["hole_1", "hole_2", "hole_3", "hole_4", "field"],
        relation="jumps_in",
        trial_seed=10,
        params={'prompt': '3', 'type': 'cot'},
        graph=graph,
        events=events
    )

In [109]:
res = sim.run_simulation(15)
print(sim.formal_to_story(res))

Alice jumps in hole_1. Charlie jumps in hole_4. Bob jumps in hole_1. Danny jumps in hole_4. Charlie jumps in hole_3. Alice jumps in hole_2. Danny jumps out to the field. Bob jumps in hole_2. Charlie jumps in hole_2. Danny jumps in hole_1. Danny jumps in hole_2. Charlie jumps in hole_3. Danny jumps in hole_3. Bob jumps in hole_3. Danny jumps in hole_2


### Experiment Setup

    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc}
    event_dict[11] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[12] = {"name": "exclusive_random", "actors": poi, "stop": 12 + mislead_distance}
    event_dict[12 + mislead_distance] = {"name": "mislead", "actors": poi}
    event_dict[12 + mislead_distance+1] = {"name": "exclusive_random", "actors": poi, "stop": story_length}
---

    

In [ ]:
df = pd.DataFrame({'Story':[], 'Label':[], 'P1':[], 'P2':[]})
df.head()

possible_people = ["Alice", "Bob", "Charlie", "Danny", "Edward", "Frank", "Georgia"]
num_people = 5
locations = ["hole_1", "hole_2", "hole_3", "hole_4", "field"]
graph = {
        "hole_1": ["hole_2", "field"],
        "hole_2": ["hole_1", "hole_3"],
        "hole_3": ["hole_2", "hole_4"],
        "hole_4": ["hole_3", "field"],
        "field": ["hole_1", "hole_4"]
    }
story_length = 21
num_trials = 5
mislead_distance = 3

random.seed(25)

for _ in range(num_trials):
    event_dict = {}
    poi = random.sample(possible_people[:num_people], 2)
    loc = random.sample(locations,3)
    #second_loc = random.choice(graph[loc])
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc[0]}
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc[1]}
    event_dict[16] = {"name":"move", "actor":poi[-1], "location": loc[2]}
    event_dict[17] = {"name": "exclusive_random", "actors": poi, "stop": 17 + mislead_distance}
    event_dict[17 + mislead_distance] = {"name": "mislead", "actors": poi}
    #event_dict[17 + mislead_distance+1] = {"name": "exclusive_random", "actors": poi, "stop": story_length}
    sim = StorySimulator(
        people=possible_people[:num_people],
        locations=["hole_1", "hole_2", "hole_3", "hole_4", "field"],
        relation="jumps_in",
        params={'prompt': '3', 'type': 'cot'},
        graph=graph,
        events=event_dict
    )
    res = sim.run_simulation(story_length)
    cp_answer = sim.current_locations[poi[1]][10]
    story = sim.formal_to_story(res)
    d = {'Story':[], 'Label':[], 'P1':[], 'P2':[]}
    d['P1'] = ",".join(poi[:-1]) if len(poi[:-1]) > 1 else poi[:-1]
    print(d["P1"])
    d['P2'] = poi[-1]
    d['Story'].append(story)
    d['Label'].append(loc[2])
    df = pd.concat([df, pd.DataFrame(d)])

['Bob']
['Alice']
['Alice']
['Alice']
['Alice']


In [122]:
df.head()

,Story,Label,P1,P2,Responses
0,Bob jumps in hole_1. Charlie jumps in hole_4. ...,hole_3,Bob,Alice,"To determine where Bob thinks Alice is, we mus..."
0,Alice jumps in hole_1. Alice jumps in hole_2. ...,field,Alice,Charlie,Alice last encountered Charlie in hole_4 befor...
0,Alice jumps in hole_4. Alice jumps in hole_3. ...,field,Alice,Charlie,"To determine where Alice thinks Charlie is, we..."
0,Bob jumps in hole_4. Alice jumps in hole_4. Al...,hole_4,Alice,Charlie,"Throughout the story, Alice has not jumped out..."
0,Alice jumps in hole_1. Charlie jumps in hole_4...,hole_4,Alice,Bob,"From the story, Alice's last known location is..."


In [ ]:
intial_prompt = f"Read the following story and answer the question at the end. Note that all characters start in the {sim.locations[-1]}."
responses = []
for _ ,row in df.iterrows():
    # row['P1'].split(',')[-1]
    prompt = f"{intial_prompt}\n{row['Story']}\nQ: Where does {row['P1']} think {row['P2']} is?"    
    answer = prompt_gpt(prompt, 'gpt-o1')
    responses.append(answer)
df['Responses'] = responses

In [124]:
print(df['Responses'].iloc[4])

From the story, Alice's last known location is hole_3. Up to the point where Bob last interacts with that hole, he was in the field after jumping in hole_4 and then jumping out to the field. Alice, being unaware of Bob's movements due to the isolated holes, thinks Bob might still be in hole_1 since that's the hole they both shared after their earliest interactions.


In [118]:
df.apply(lambda x: x['Label'] in x['Responses'], axis=1)

0    True
0    True
0    True
0    True
0    True
dtype: bool

In [117]:
for i ,row in df.iterrows():
    print("\n".join(row['Story'].split(".")))
    print('---')

Bob jumps in hole_1
 Charlie jumps in hole_4
 Charlie jumps out to the field
 Charlie jumps in hole_1
 Charlie jumps out to the field
 Alice jumps in hole_1
 Charlie jumps in hole_4
 Charlie jumps in hole_3
 Bob jumps in hole_2
 Charlie jumps in hole_2
 Alice jumps in hole_2
 Alice jumps in hole_3
 Charlie jumps in hole_3
 Charlie jumps in hole_2
 Charlie jumps in hole_1
 Alice jumps in hole_4
 Charlie jumps out to the field
 Charlie jumps in hole_4
 Charlie jumps out to the field
 Charlie jumps in hole_1
 Charlie jumps out to the field
---
Alice jumps in hole_1
 Alice jumps in hole_2
 Charlie jumps in hole_1
 Alice jumps in hole_1
 Bob jumps in hole_1
 Alice jumps out to the field
 Bob jumps out to the field
 Bob jumps in hole_4
 Alice jumps in hole_4
 Bob jumps out to the field
 Charlie jumps in hole_4
 Charlie jumps out to the field
 Bob jumps in hole_4
 Bob jumps out to the field
 Bob jumps in hole_1
 Charlie jumps in hole_1
 Bob jumps out to the field
 Bob jumps in hole_1
 Bob jum